In [1]:
import pandas as pd
import numpy as np
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch import Trainer
from lightning.pytorch import loggers as pl_loggers

from library.taxonomy_constructors import SyntheticTaxonomy, CrossPredictionsTaxonomy
from library.models import ResNetModel
from library.datasets import Country211MappedDataModule, Country211Mapped

# Load Country211 dataset information
country211_dataset = Country211Mapped(root="datasets", download=False)
country211_labels = country211_dataset.classes
print(f"Total Country211 classes: {len(country211_labels)}")

# Create a simple target mapping for Country211 (0 to num_classes-1)
country211_targets = pd.DataFrame(
    {"class_id": range(len(country211_labels)), "class_name": country211_labels}
)

# Reduce the precision of matrix multiplication to speed up training
torch.set_float32_matmul_precision("medium")

Total Country211 classes: 211


In [2]:
# Create synthetic taxonomy with 2 domains for Country211
synthetic_taxonomy = SyntheticTaxonomy.create_synthetic_taxonomy(
    num_atomic_concepts=len(country211_labels),
    num_domains=2,
    domain_class_count_mean=150,  # Adjusted for Country211's 211 classes
    domain_class_count_variance=10,
    concept_cluster_size_mean=3,
    concept_cluster_size_variance=1,
    no_prediction_class=True,
    atomic_concept_labels=country211_labels,
    relationship_type="true",
)

# Extract domain mappings
domain_A_mapping = synthetic_taxonomy.domains[0].to_mapping()
domain_B_mapping = synthetic_taxonomy.domains[1].to_mapping()

print(f"Domain A classes: {len(set(domain_A_mapping.values()))}")
print(f"Domain B classes: {len(set(domain_B_mapping.values()))}")
print(f"Domain A original classes: {len(domain_A_mapping)}")
print(f"Domain B original classes: {len(domain_B_mapping)}")

Domain A classes: 52
Domain B classes: 49
Domain A original classes: 152
Domain B original classes: 152


In [ ]:
# Configuration
TRAIN = True  # Set to True to train models from scratch


def train_domain_model(domain_mapping, domain_name, logger_name, model_name):
    """Train a ResNet model for a specific domain using Country211"""
    tb_logger = pl_loggers.TensorBoardLogger(save_dir="logs", name=logger_name)
    dataset = Country211MappedDataModule(mapping=domain_mapping, split="train")

    trainer = Trainer(
        max_epochs=60,
        logger=tb_logger if TRAIN else False,
        callbacks=[
            ModelCheckpoint(
                dirpath="checkpoints",
                monitor="val_loss",
                mode="min",
                save_top_k=1,
                filename=model_name,
                enable_version_counter=False,
            )
        ],
    )

    if TRAIN:
        model = ResNetModel(
            num_classes=len(set(domain_mapping.values())),
            architecture="resnet50",
            optim="sgd",
            optim_kwargs={
                "lr": 0.01,
                "momentum": 0.9,
                "weight_decay": 5e-4,
            },
        )
        trainer.fit(model, datamodule=dataset)
        results = trainer.test(datamodule=dataset, ckpt_path="best")
    else:
        model = ResNetModel.load_from_checkpoint(f"checkpoints/{model_name}.ckpt")
        results = trainer.test(model, datamodule=dataset)

    print(f"{domain_name} Results: {results}")
    return results

In [4]:
# Train Domain A model
print("Training/Testing Domain A Model:")
domain_A_results = train_domain_model(
    domain_A_mapping,
    "Domain A",
    "country211_synthetic_A",
    "resnet50-country211-synthetic-A-min-val-loss",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Training/Testing Domain A Model:


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 357/357 [01:50<00:00,  3.24it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      eval_accuracy         0.8900877237319946
        eval_loss           0.3789011836051941
        hp_metric           0.8900877237319946
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Domain A Results: [{'eval_loss': 0.3789011836051941, 'eval_accuracy': 0.8900877237319946, 'hp_metric': 0.8900877237319946}]


In [ ]:
# Train Domain B model
print("Training/Testing Domain B Model:")
domain_B_results = train_domain_model(
    domain_B_mapping,
    "Domain B",
    "country211_synthetic_B",
    "resnet50-country211-synthetic-B-min-val-loss",
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Training/Testing Domain B Model:


/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/sentinel/Development/master-thesis/project/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | ResNet           | 26.3 M | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
26.3 M    Trainable params
0         Non-trainable params
26.3 M    Total params
105.206   Total estimated model params size (MB)
162       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 7:   7%|▋         | 25/357 [00:09<02:02,  2.71it/s, v_num=2] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
# Configuration for prediction generation
PREDICT = True  # Set to True to generate predictions from scratch

if PREDICT:
    # Load datasets
    dataset_domain_A = Country211MappedDataModule(
        mapping=domain_A_mapping, split="test"
    )
    dataset_domain_B = Country211MappedDataModule(
        mapping=domain_B_mapping, split="test"
    )

    # Load trained models
    model_domain_A = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet50-country211-synthetic-A-min-val-loss.ckpt"
    )
    model_domain_A.eval()

    model_domain_B = ResNetModel.load_from_checkpoint(
        "checkpoints/resnet50-country211-synthetic-B-min-val-loss.ckpt"
    )
    model_domain_B.eval()

    trainer = Trainer(logger=False, enable_checkpointing=False)

    # Generate cross-domain predictions
    print("Generating cross-domain predictions...")
    model_A_on_domain_B = trainer.predict(model_domain_A, datamodule=dataset_domain_B)
    model_B_on_domain_A = trainer.predict(model_domain_B, datamodule=dataset_domain_A)

    # Convert predictions to class indices
    predictions_A_on_B = torch.cat(model_A_on_domain_B).argmax(dim=1)  # type: ignore
    predictions_B_on_A = torch.cat(model_B_on_domain_A).argmax(dim=1)  # type: ignore

    # Get ground truth targets
    domain_A_targets = torch.cat(
        [label for _, label in dataset_domain_A.predict_dataloader()]
    )
    domain_B_targets = torch.cat(
        [label for _, label in dataset_domain_B.predict_dataloader()]
    )

    # Save predictions
    pd.DataFrame(
        {
            "domain_A": domain_A_targets,
            "predictions_B_on_A": predictions_B_on_A,
        }
    ).to_csv("data/country211_2domain_A_predictions.csv", index=False)

    pd.DataFrame(
        {
            "domain_B": domain_B_targets,
            "predictions_A_on_B": predictions_A_on_B,
        }
    ).to_csv("data/country211_2domain_B_predictions.csv", index=False)

    print("Predictions saved to CSV files.")

# Load prediction results
df_A = pd.read_csv("data/country211_2domain_A_predictions.csv")
df_B = pd.read_csv("data/country211_2domain_B_predictions.csv")

print(f"Domain A predictions shape: {df_A.shape}")
print(f"Domain B predictions shape: {df_B.shape}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/sentinel/.conda/envs/master-thesis/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Generating cross-domain predictions...
Predicting DataLoader 0: 100%|██████████| 238/238 [01:24<00:00,  2.82it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Predicting DataLoader 0: 100%|██████████| 238/238 [01:18<00:00,  3.03it/s]

Predictions saved to CSV files.
Domain A predictions shape: (15200, 2)
Domain B predictions shape: (15200, 2)
Predictions saved to CSV files.
Domain A predictions shape: (15200, 2)
Domain B predictions shape: (15200, 2)


In [ ]:
# Construct taxonomy from cross-domain predictions
taxonomy = CrossPredictionsTaxonomy.from_cross_domain_predictions(
    cross_domain_predictions=[
        (0, 1, np.array(df_B["predictions_A_on_B"], dtype=np.intp)),
        (1, 0, np.array(df_A["predictions_B_on_A"], dtype=np.intp)),
    ],
    domain_targets=[
        (0, np.array(df_A["domain_A"], dtype=np.intp)),
        (1, np.array(df_B["domain_B"], dtype=np.intp)),
    ],
    domain_labels=synthetic_taxonomy.domain_labels,
    relationship_type="hypothesis",
)

print("Taxonomy constructed from cross-domain predictions.")

Taxonomy constructed from cross-domain predictions.


In [ ]:
# Generate and save taxonomy visualizations
print("Generating taxonomy visualizations...")

taxonomy.visualize_graph("2-Domain Country211 Model Taxonomy").save_graph(
    "output/country211_2domain_synthetic_model_taxonomy.html"
)

synthetic_taxonomy.visualize_graph(
    "2-Domain Country211 Ground Truth Taxonomy"
).save_graph("output/country211_2domain_synthetic_taxonomy.html")

print("Taxonomy visualizations saved to output/ directory.")

Generating taxonomy visualizations...
Taxonomy visualizations saved to output/ directory.


In [ ]:
# Evaluate taxonomy against ground truth
edr = taxonomy.edge_difference_ratio(synthetic_taxonomy)
precision, recall, f1 = taxonomy.precision_recall_f1(synthetic_taxonomy)

print("2-Domain Country211 Taxonomy Evaluation:")
print(f"Edge Difference Ratio: {edr:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

2-Domain Country211 Taxonomy Evaluation:
Edge Difference Ratio: 0.9771
Precision: 0.0396
Recall: 0.0179
F1 Score: 0.0246


In [ ]:
# Build universal taxonomies
print("Building universal taxonomies...")

taxonomy.build_universal_taxonomy()
taxonomy.visualize_graph("2-Domain Country211 Model Universal Taxonomy").save_graph(
    "output/country211_2domain_synthetic_model_universal_taxonomy.html"
)

synthetic_taxonomy.build_universal_taxonomy()
synthetic_taxonomy.visualize_graph(
    "2-Domain Country211 Ground Truth Universal Taxonomy"
).save_graph("output/country211_2domain_synthetic_universal_taxonomy.html")

print("Universal taxonomy visualizations saved to output/ directory.")

Building universal taxonomies...
Universal taxonomy visualizations saved to output/ directory.
Universal taxonomy visualizations saved to output/ directory.
